# Rank Most Actors By Genre

Using various distance metrics, identify the most similar actors to a given query actor by the genres in which they've starred.

In [1]:
%matplotlib inline


In [2]:
import pandas as pd

import json

In [15]:
actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}


with open("../data/imdb_movies_2000to2022.prolific.json", "r") as in_file:
    for line in in_file:
        
        # Read the movie on this line and parse its json
        this_movie = json.loads(line)
                    
        # Add all actors to the id->name map
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        # For each actor, add this movie's genres to that actor's list
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            # Increment the count of genres for this actor
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            # Update the map
            actor_genre_map[actor_id] = this_actors_genres
            
        # Finished with this film
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"]
        })

In [16]:
# Check the output for actor Hugh Jackman, actor ID nm0413168
actor_genre_map['nm0413168']

{'Comedy': 7,
 'Fantasy': 3,
 'Romance': 5,
 'Action': 14,
 'Adventure': 11,
 'Sci-Fi': 10,
 'Crime': 6,
 'Thriller': 2,
 'Animation': 4,
 'Drama': 12,
 'Mystery': 5,
 'Biography': 4,
 'Musical': 2,
 'History': 1}

In [18]:
# Get all actors as an index for a dataframe
index = actor_genre_map.keys()

# Get the genre-counts for each actor in the index
rows = [actor_genre_map[k] for k in index]

# Create the data frame from these rows, with the actors as index
df = pd.DataFrame(rows, index=index)

# Fill NAs with zero, as NA means the actor has not starred in that genre
df = df.fillna(0)

df

,Comedy,Fantasy,Romance,Drama,Mystery,Thriller,Action,Biography,Crime,War,...,Horror,Documentary,Sport,News,Family,Music,,Western,Short,Reality-TV
nm0000212,7.0,1.0,6.0,6.0,1.0,2.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0413168,7.0,3.0,5.0,12.0,5.0,2.0,14.0,4.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0000630,8.0,2.0,6.0,14.0,2.0,3.0,4.0,5.0,1.0,1.0,...,3.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0005227,10.0,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
nm0864851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nm9504284,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm10592896,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm7216750,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nm0936300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [22]:
#Setting the actor we will be comparing to
target_actor_id = 'nm1165110'

In [27]:
#Gathering the genres for that actor
target_actor_ratings = df.loc[target_actor_id]

#Generating distances from that actor to all the others
distances = scipy.spatial.distance.cdist(df, [target_actor_ratings], metric="cosine")[:,0]

query_distances = list(zip(df.index, distances))

In [28]:
#Printing the top ten most similar actors to our target
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    similar_actor = actor_genre_map[similar_actor_id]
    print(similar_actor_id, actor_name_map[similar_actor_id], similar_genre_score)


nm1165110 Chris Hemsworth 0.0
nm0000129 Tom Cruise 0.026884630613554727
nm0147147 Henry Cavill 0.04459907371977079
nm0829032 Ray Stevenson 0.045203363012771614
nm5899377 Tiger Shroff 0.04878254078965638
nm1679372 Sudeep 0.051694237969527324
nm0003244 Jordi Mollà 0.053414613481432394
nm0636280 Richard Norton 0.05652570932413603
nm0607884 Mark Mortimer 0.05652570932413603
nm2018237 Taylor Kitsch 0.05800371753039013


In [31]:
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_distances

In [32]:
# dist = DistanceMetric.get_metric("euclidean")
dist = cosine_distances

In [35]:
#Gathering the genres for that actor
target_actor_ratings = df.loc[target_actor_id]

#Generating distances from that actor to all the others
# distances = dist.pairwise(df, [target_actor_ratings])[:,0]
distances = dist(df, [target_actor_ratings])[:,0]

query_distances = list(zip(df.index, distances))

#Printing the top ten most similar actors to our target
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
    similar_actor = actor_genre_map[similar_actor_id]
    print(similar_actor_id, actor_name_map[similar_actor_id], similar_genre_score)



nm1165110 Chris Hemsworth 0.0
nm0000129 Tom Cruise 0.026884630613554727
nm0147147 Henry Cavill 0.04459907371977079
nm0829032 Ray Stevenson 0.045203363012771614
nm5899377 Tiger Shroff 0.04878254078965649
nm1679372 Sudeep 0.051694237969527324
nm0003244 Jordi Mollà 0.053414613481432394
nm0636280 Richard Norton 0.05652570932413603
nm0607884 Mark Mortimer 0.05652570932413603
nm2018237 Taylor Kitsch 0.05800371753039024
